In [ ]:
from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
import pygame

def evolution_controller():
    sensors = SensorSet([
        GenomeBinarySensor(genome_id=8, draw=False),
        GenomeBinarySensor(genome_id=9, draw=False)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
        seed=None,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=((1/3) * np.pi), _min=-((2/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
        GeneRule(_max=((2/3) * np.pi), _min=-((1/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
    ]

    phenotype = [
        AverageSpeedBehavior(),
        AngularMomentumBehavior(),
        RadialVarianceBehavior(),
        ScatterBehavior(),
        GroupRotationBehavior(),
        # GeneElementDifference(8, 9)
    ]

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        seed=None,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=50,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=2500,
        display_novelty=True,
        save_archive=True,
        show_gui=True,
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = ModifiedHaltingEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution, screen

def build_random_dataset():
    halted_evolution, screen = evolution_controller()
    baseline_data = DataBuilder("data/full-dual-sensors", steps=1200, agents=30, ev=halted_evolution, screen=screen)
    baseline_data.create()
    baseline_data.evolution.close()

# import time
# t_1 = time.time()
# build_random_dataset()
# print(f"TIME TO DATASET: {time.time() - t_1}")

# Embed into Latent Space

In [ ]:

from generation.evolution import ModifiedHaltingEvolution
from scipy import ndimage
import random

def batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.batch_network_from_numpy(anchor_images, pos_images, neg_images)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image, with_transforms=False):
    optim.zero_grad()
    if with_transforms:
        anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    else:
        anchor_out, pos_out, neg_out = network.network_from_numpy(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def pretraining(data, network, loss_fn, optim, data_cutoff=None, data_size=500):
    if data_cutoff is None:
        data_cutoff = len(data) - 1
    samples = np.random.random_integers(0, data_cutoff, (data_size, 2))
    total_loss = 0
    total_updates = 0
    for i, s in enumerate(samples):
        if i % (data_size // 20) == 0:
            print(f"Unsupervised Training.. {(i * 100) / data_size}")
        anchor_image = data[s[0]][0]
        pos_image = data[s[0]][0]

        pos_images = np.stack([
            [ndimage.rotate(pos_image, 90)],
            [ndimage.rotate(pos_image, 180)],
            [ndimage.rotate(pos_image, 270)],
            # ndimage.gaussian_filter(pos_image, 1),
            # ndimage.gaussian_filter(pos_image, 3),
            # ndimage.gaussian_filter(pos_image, 5),
        ])

        anchor_images = np.stack([[anchor_image] for _ in pos_images])
        neg_images = np.stack([ [data[random.randint(0, data_cutoff)][0]] for _ in pos_images])

        loss = batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images)
        total_loss += loss.item()
        total_updates += 1

    return total_loss / total_updates


def train(network, loss_fn, optim, anchor_img, pos_img, neg_img):

    pos_images = np.stack([
        [ndimage.rotate(pos_img, 90)],
        [ndimage.rotate(pos_img, 180)],
        [ndimage.rotate(pos_img, 270)],
        # ndimage.gaussian_filter(pos_img, 1),
        # ndimage.gaussian_filter(pos_img, 3),
        # ndimage.gaussian_filter(pos_img, 5),
    ])

    anchor_images = np.stack([[anchor_img] for _ in pos_images])
    neg_images = np.stack([[neg_img] for _ in pos_images])

    loss = batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images)
    return loss

def human_in_the_loop(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at):
    print("HIL TIME!")
    improvements, human_loss, triplet_helpfulness, embedded_archive = hil.humanInput(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at)
    print(f"Improvement Count: {improvements}, loss: {human_loss}")

    HIL_archive.save_to_file(f"data/queries/{trial_name}_hil.csv")
    random_archive.save_to_file(f"data/queries/{trial_name}_rand.csv")
    return improvements, human_loss, triplet_helpfulness, embedded_archive

In [ ]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import numpy as np
import time
from torch.utils.tensorboard import SummaryWriter

trial_name = f"{str(int(time.time()))}"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding(out_size=15).to(device)

TRAIN = True
SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
CLUSTER_AND_DISPLAY = True
CONCAT_BVEC_AND_EVEC = True
PASS_THROUGHS = 1
RESULTS_DATA_SIZE = 10000
EVOLUTIONS_PER_EPOCH = 50
K_CLUSTERS = 8
INITIAL_EVOLVES = 3

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-dual-sensors", rank=0)
# evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=180, n_agents=12, evolve_population=50, seed=None)
evolution, _ = evolution_controller()
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)

loss_fn = torch.nn.TripletMarginLoss(margin=15)
hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=RESULTS_DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()
random_archive = DataAggregationArchive(scalar=True)
EPSILON = 0.5

simulation_time = 0
evolution_time = 0
training_time = 0
hil_time = 0


# TODO: Add Randomly-sampled Contrastive/Triplet Loss
if PRETRAINING:
    network.load_model("two_sensor_subz1")
    print("Pretrained model loaded!")

dataset = anchor_dataset if EVOLUTION else sampled_dataset

if TRAIN:
    writer = SummaryWriter()
    STOP_FLAG = False
    for epoch in range(PASS_THROUGHS + 1):
        if EVOLUTION:
            start_time = time.time()
            evolution.restart_screen()
            for gen in range(EVOLUTIONS_PER_EPOCH if epoch > 0 else INITIAL_EVOLVES):
                # Simulate current population + Save Data
                for i in range(len(evolution.getPopulation())):
                    # The collection of the original behavior vector below is only used to collect data to compare with the baseline
                    visual_behavior, genome, baseline_behavior = evolution.next()
                    dataset.new_entry(visual_behavior, genome, baseline_behavior)
                simulation_time += (time.time() - start_time)

                # Then, evolve
                start_time = time.time()
                embedded_archive = hil.getEmbeddedArchive(dataset, network, concat_behavior=CONCAT_BVEC_AND_EVEC)
                evolution.overwriteArchive(embedded_archive, random_archive)
                embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
                evolution.overwriteBehavior(embedded_behavior)
                evolution.evolve()
                evolution.restart_screen()
                evolution_time += (time.time() - start_time)

                print(f"Evolution complete for e{epoch} and gen{gen}")
                print(f"Size of Evolved Archive: {len(evolution.behavior_discovery.archive.archive)}")

                # Record the accuracy of the medoids with respect to the synthetic policy
                medoid_acc, cluster_acc = 0, 0
                if len(dataset) > 0 and SAVE_CLUSTER_MEDOIDS:
                    if SYNTHETIC_HIL and EVOLUTION:
                        hil.synthetic_knowledge = hil.syntheticBehaviorSpace(dataset)
                        print(f"Synthetic Human Knowledge Size: {len(hil.synthetic_knowledge.labels_)}")
                    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

                # Cluster current dataset, display clusters, and save for analysis
                if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
                    hil.embed_and_cluster(network, dataset, auto_quit=True)
                    evolution.restart_screen()

                evolution.saveArchive(trial_name)

        # Human in the Loop determines behavior embedding
        if HUMAN_IN_LOOP and epoch < PASS_THROUGHS:
            start_time = time.time()
            improvements, human_loss, triplet_helpfulness, _ =   human_in_the_loop(
                                                                dataset,
                                                                network,
                                                                optimizer,
                                                                loss_fn,
                                                                HIL_archive,
                                                                random_archive,
                                                                len(dataset)
                                                            )
            hil_time += (time.time() - start_time)

        # Train on past user information
        if TRAIN:
            start_time = time.time()
            loss = None

            average_loss = 50
            loop = 0
            BATCH_SIZE = 5
            network.train()
            while (average_loss > 0.01) and loop < 20:
                loss_sum = 0
                total_loss = 0
                for i, (anchor, pos, neg) in enumerate(HIL_archive):
                    anchor_encoding = dataset[anchor][0]
                    similar_encoding = dataset[pos][0]
                    anti_encoding = dataset[neg][0]
                    loss = train(network, loss_fn, optimizer, anchor_encoding, similar_encoding, anti_encoding)
                    loss_sum += loss.item()
                    total_loss += 1

                    if loss is not None and i > 0 and i % 20 == 0:
                        print(f"Epoch Progress: {(i*100) / len(HIL_archive)}%, Immediate Loss: {loss.item()}")

                average_loss = loss_sum / (total_loss + 1)
                print(f"Loop: {loop}, Average Loss: {average_loss}")
                loop += 1

            print(f"Epoch: {epoch}")
            training_time += (time.time() - start_time)

            writer.add_scalar("Loss/Average", loss_sum / (total_loss + 1), epoch)

        if SAVE_CLUSTER_MEDOIDS:
            writer.add_scalar("Accuracy/MedoidClassification", medoid_acc, epoch)
            writer.add_scalar("Accuracy/RandomSampleClassification", cluster_acc, epoch)

        if HUMAN_IN_LOOP:
            writer.add_scalar("Loss/TripletQuality", triplet_helpfulness, epoch)
            writer.add_scalar("Queries/Total_Human_Queries", epoch*8*9, epoch)
            writer.add_scalar("Queries/Total_Triplets_Generated", len(HIL_archive), epoch)
            writer.add_scalar("Queries/Total_Random_Classes", len(random_archive), epoch)

        if EVOLUTION:
            writer.add_scalar("Novelty/Highest", evolution.behavior_discovery.getBestScore(), epoch)
            writer.add_scalar("Novelty/Average", evolution.behavior_discovery.getAverageScore(), epoch)

        writer.add_scalar("Time/Simulation", simulation_time, epoch)
        writer.add_scalar("Time/HIL", hil_time, epoch)
        writer.add_scalar("Time/Evolution", evolution_time, epoch)
        writer.add_scalar("Time/Training", training_time, epoch)

        scheduler.step()

evolution.close()

In [ ]:
# Evolve and Display?
print(len(evolution.archive.archive))

In [ ]:
import pygame
pygame.quit()

# Pretraining

In [ ]:
# Save Model
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive

PRETRAINING = True
target = 0.01
loss = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding(out_size=15).to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)
sampled_dataset = SwarmDataset("data/full-dual-sensors", rank=0)

t_1 = time.time()
if PRETRAINING:
    epochs = 0
    while loss > target:
        loss = pretraining(sampled_dataset, network, loss_fn, optimizer, data_cutoff=None, data_size=1000)
        print(f"Epoch {epochs}, loss: {loss}")
        epochs += 1
        scheduler.step()

print(f"Total Pre-training Time: {time.time() - t_1}")

network.save_model()

In [ ]:
network.save_model()

In [ ]:
print(optimizer.lr)

In [ ]:

import torch
from networks.embedding import NoveltyEmbedding
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from data.swarmset import DataBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
network.load_model("trialA-10-18-2022")
# anchor_dataset = SwarmDataset("data/full", rank=0)

WRITE_OUT = True
if WRITE_OUT:
    network.eval()
    test_archive = NoveltyArchive()
    for i in range(len(anchor_dataset)):
        anchor_encoding, genome, _ = anchor_dataset[i]
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        test_archive.addToArchive(embedding, genome)

In [ ]:
from ui.clustering_gui import ClusteringGUI
import pygame

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=test_archive, k_clusters=6, world_config=world_config, tsne_perplexity=16, tsne_early_exaggeration=12, skip_tsne=False)
gui = ClusteringGUI(config)
gui.displayGUI()
# results(config)
pygame.quit()

# Clustering + Analysis

In [ ]:
from ui.clustering_gui import ClusteringGUI

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(human_l_hist, "b", label='Human Loss')
plt.ylabel("Loss")
plt.xlabel("Time (Epochs)")
plt.legend()